In [49]:
import tools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import plot_roc_curve, classification_report
from confusion_matrix.confusion_matrix import plot_confusion_matrix_from_data
import joblib

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.random_projection import SparseRandomProjection

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [50]:
from importlib import reload
reload(tools)
directory = '../data/data_final'
database = '../data/data_final/database.csv'

X, y = tools.getDataTransformed(database, directory)
X_train_origin, X_test_origin, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [51]:
dimention_reduction = [PCA, TruncatedSVD, LocallyLinearEmbedding, SparseRandomProjection] # LinearDiscriminantAnalysis,
techniques = [RandomForestClassifier(), KNeighborsClassifier(), SVC(), AdaBoostClassifier(), XGBClassifier(use_label_encoder=False, eval_metric='logloss')]
grid_params =[
    {
        'n_estimators': np.arange(10, 511, 100),
        'criterion': ['gini', 'entropy']
    },
    {
        'n_neighbors': np.arange(1,20, 3),
        'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    {
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'degree' : np.arange(2,5)
    },
    {
        'n_estimators' : np.arange(10, 70, 10)
    },
    {
        'booster' : ['gbtree', 'gblinear', 'dart'],
        'eta' : np.arange(0.1,0.9,0.2)
    }
]

data_size = [0] + list(range(50,501,200))

TypeError: can only concatenate list (not "range") to list

In [ ]:
class MachineLearningTechnique():
    def __init__(self, model, reduction):
        self.model = model
        self.reduction = reduction

    def score(self, X_test, y_test): 
        self.score = self.test_score = self.model.score(X_test, y_test) # changer par la fonction qui permet de discriminer les models
        return self.score 

    def save(self):
        joblib.dump(best_model,'./model.sav')
        if self.reduction is not None:
            joblib.dump(self.reduction, './reduction.sav')

    def displayMetrics(self, X_test, y_test):
        X_test = reduction.transform(X_test)
        print('Best score : ', self.score)
        print('Best reduction : ', self.reduction)
        if self.reduction is not None:
            print('Best size : ', self.reduction.n_component)
        predictions = self.model.predict(X_test)
        print('Model : ', self.model)
        sk_report = classification_report(
            digits=6,
            y_true=y_test, 
            y_pred=predictions
        )
        print(sk_report)
        plot_confusion_matrix_from_data(y_test, predictions,columns=['non normal','normal'])
        plot_roc_curve(self.model, X_test, y_test)


In [ ]:
"""technique = RandomForestClassifier()
red = None
print(grid_params[0])
grid = GridSearchCV(estimator=technique, param_grid=grid_params[0])
grid.fit(X_train_origin, y_train)
print(grid.best_estimator_)
model = MachineLearningTechnique(grid.best_estimator_, red)
model.displayMetrics(X_test_origin, y_test)"""

'technique = RandomForestClassifier()\nred = None\nprint(grid_params[0])\ngrid = GridSearchCV(estimator=technique, param_grid=grid_params[0])\ngrid.fit(X_train_origin, y_train)\nprint(grid.best_estimator_)\nmodel = MachineLearningTechnique(grid.best_estimator_, red)\nmodel.displayMetrics(X_test_origin, y_test)'

In [ ]:
best_model = None
best_score = -1

for reduction in dimention_reduction:
    print(reduction)
    for size in data_size:
        print(size)
        X_train = X_train_origin.copy()
        X_test = X_test_origin.copy()
        red = None
        if size != 0: # cas ou l'on fait une reduction de dimension
            red = reduction(n_components=size)
            try:
                X_train = red.fit_transform(X_train, y_train)
            except Exception as e:
                print(e)
                print(reduction)
                print('fin')
                continue
            X_test = red.transform(X_test)
        for i in range(len(techniques)):
            print(techniques[i])
            grid = GridSearchCV(estimator=techniques[i], param_grid=grid_params[i])
            grid_result = grid.fit(X_train, y_train)
            model = MachineLearningTechnique(grid_result.best_estimator_, red)
            test_score = model.score(X_test, y_test)
            if test_score > best_score:
                best_model = model
                best_score = test_score
                
best_model.displayMetrics(X_test, y_test)
best_model.save()

<class 'sklearn.decomposition._pca.PCA'>
0
RandomForestClassifier()
KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)
200
RandomForestClassifier()


C:\Users\jleva\anaconda3\envs\GCMS\lib\site-packages\sklearn\decomposition\_pca.py:561: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ / total_var.sum()


KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)
400
RandomForestClassifier()


C:\Users\jleva\anaconda3\envs\GCMS\lib\site-packages\sklearn\decomposition\_pca.py:561: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ / total_var.sum()


KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>
0
RandomForestClassifier()
KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None,

C:\Users\jleva\anaconda3\envs\GCMS\lib\site-packages\sklearn\decomposition\_truncated_svd.py:205: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)
400


C:\Users\jleva\anaconda3\envs\GCMS\lib\site-packages\sklearn\decomposition\_truncated_svd.py:205: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


RandomForestClassifier()
KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)
<class 'sklearn.manifold._locally_linear.LocallyLinearEmbedding'>
0
RandomForestClassifier()
KNeighborsClassifier()
SVC()
AdaBoostClassifier()
XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
     

ValueError: X has 400 features, but DecisionTreeClassifier is expecting 504 features as input.

Comparaison avec un Humain (Test non complet pour le moment)
bertrandp-20190201 : 3 + 4 -> 4<br>
settouchei-20190424 : 2 + 3 -> 2<br>
mangonim-20190717 : 0 -> 0<br>
syh-20190912 : 2 -> 2<br>
mastourab-20190219 : 1 (+2 ?) -> 1;4;2<br>
chesinara-20191009 : 3 -> 3<br>
husica-20191108 : 1 + 3 + 4 -> 1<br>
bruyelleCn-20191017 : 3 -> 3<br>
dimolad-20190909 : 2 -> 2<br>
botafemmin-20190225 : 1 -> 1<br>
colakeram-20190923 : 0 ? (sinon 2) -> 0<br>
heriquem-20190329 : 4 ? (cinnamate) -> 5<br>
durmc-20190724 : 0 ? -> 4<br>
piechockil-20190910 : 1 + 4 -> 3;4<br>
heriquem-20190329 : 3 -> 5<br>
genouxm-20190401 : 2 + 3 -> 2<br>

In [ ]:
y_test = [0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0]
y_predict = [0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0]

sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=y_predict
)
print(sk_report)
plot_confusion_matrix_from_data(y_test, y_predict,columns=['non normal','normal'])

Error: Session cannot generate requests